# Calculate the distance from driving to cbd
### The main function of the code is to calculate the driving distance and time to Melbourne CBD for each property in the rental dataset by calling the OpenRoute Service API and storing the results in the dataset, which is then saved as a CSV file. This process has a flow-limiting mechanism added to the code in order to comply with the API's request restrictions.

In [ ]:
import requests
import pandas as pd
import time
import os

# Load rental data
rent_df = pd.read_csv("../../data/landing/cleaned_rent_data.csv")

# Add new columns for storing results
rent_df['CBD_distance'] = None
rent_df['CBD_duration_time'] = None

# Coordinates of Melbourne CBD
CBD_longti = 144.9627651963711
CBD_latiti = -37.810045355688764

# API request limitations
max_per_Min = 40
time_sleep = 60 / max_per_Min

# User input API key and batch range
your_api_key = os.getenv('ORS_API_KEY')  # Use an environment variable to store the API key
start = 1
end = 2000

# Ensure the save file path exists
output_dir = '../../data/External_data/'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Iterate over the specified index range of the rental dataset
for index, row in rent_df.iloc[start-1:end].iterrows():  # Note iloc starts at 0, so start-1
    if pd.isna(row['CBD_distance']):  # Only process records where CBD_distance is empty
        longi = row['longitude']
        latit = row['latitude']

        # Construct API request URL
        api_url = f'https://api.openrouteservice.org/v2/directions/driving-car?api_key={your_api_key}&start={longi},{latit}&end={CBD_longti},{CBD_latiti}'
        response = requests.get(api_url)

        # Check request status
        if response.status_code == 200:
            info = response.json()
            features = info.get('features')
            summary = features[0]['properties']['summary']  # Get driving route information
            distance = summary['distance']  # Distance, in meters
            duration_time = summary['duration']  # Driving time, in seconds

            # Update rent_df with CBD_distance and CBD_duration_time
            rent_df.at[index, 'CBD_distance'] = distance
            rent_df.at[index, 'CBD_duration_time'] = duration_time
        else:
            print(f"Request failed with status code: {response.status_code}")

        # Limit the API request rate
        time.sleep(time_sleep)

# Save the batch of data processed by the current member
output_file = f'{output_dir}rent_data_CBD_{start}_to_{end}.csv'
rent_df.iloc[start-1:end].to_csv(output_file, index=False)

print(f'Data from row {start} to {end} saved to {output_file}')